Modeste application pour la prediction de cancer du sein en utilisant le SVM pour l'entrainement de notre modèle realisée par RAHMANI Nour el houda 

In [1]:
import pandas as pd
import sklearn

import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC


df = pd.read_csv("data.csv")
df= df.drop(['Unnamed: 32','id'],axis=1)


X = df.drop('diagnosis',axis=1).values
y = df['diagnosis'].values


x_train, x_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=0)

sc = StandardScaler()
X_train = sc.fit_transform(x_train)
X_test= sc.transform(x_test)


svc = SVC(probability=True)
svc.fit(X_train,y_train)


joblib.dump(sc, 'scaler.pkl')
joblib.dump(svc, 'SVM.pkl')

['SVM.pkl']

In [8]:
import tkinter as tk
from tkinter import filedialog
import numpy as np
import joblib
from sklearn.preprocessing import StandardScaler

# Charger le modèle pré-entraîné et le scaler
svc = joblib.load('SVM.pkl')
scaler = joblib.load('scaler.pkl')

# Définir les caractéristiques
features = [
    'radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean', 'smoothness_mean',
    'compactness_mean', 'concavity_mean', 'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
    'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se', 'compactness_se',
    'concavity_se', 'concave points_se', 'symmetry_se', 'fractal_dimension_se',
    'radius_worst', 'texture_worst', 'perimeter_worst', 'area_worst', 'smoothness_worst',
    'compactness_worst', 'concavity_worst', 'concave points_worst', 'symmetry_worst', 'fractal_dimension_worst'
]

def predict_cancer():
    try:
        # Collecter les données d'entrée depuis le formulaire
        input_data = [float(entry.get()) for entry in entries]
        
        # Transformer les données d'entrée à l'aide du scaler
        input_data = scaler.transform([input_data])
        
        # Faire la prédiction
        prediction = svc.predict(input_data)
        prediction_prob = svc.predict_proba(input_data)
        
        # Afficher le résultat
        if prediction[0] == 1:
            result = "Malignant"
        else:
            result = "Benign"
        
        result_label.config(text=f"Prediction Result: {result}")
    except Exception as e:
        error_label.config(text=f"Error: {str(e)}")

def load_values_from_file():
    try:
        # Ouvrir une boîte de dialogue pour sélectionner un fichier
        file_path = filedialog.askopenfilename(filetypes=[("Text files", "*.txt"), ("CSV files", "*.csv")])
        if not file_path:
            return  # L'utilisateur a annulé la sélection du fichier
        
        with open(file_path, "r") as file:
            lines = file.readlines()
            for line in lines:
                feature_name, value = line.strip().split(": ")
                # Recherche de l'index de la caractéristique
                index = features.index(feature_name)
                # Insérer la valeur dans le champ d'entrée correspondant
                entries[index].delete(0, tk.END)  # Effacer le contenu précédent
                entries[index].insert(0, value)   # Insérer la nouvelle valeur
    except Exception as e:
        error_label.config(text=f"Error loading values from file: {str(e)}")

# Créer la fenêtre principale
root = tk.Tk()
root.title("Cancer Prediction")
root.geometry("950x500")
root.configure(bg="white")  # Fond blanc

# Configuration de la police en gras
bold_font = ("TkDefaultFont", 10, "bold")
bold_font1 = ("TkDefaultFont", 12, "bold")
pink_font_color = "#DA0C81"  # Couleur rose pour le texte

title_label = tk.Label(root, text="Empowering Early Detection: A Breast Cancer Awareness-Based Application", font=bold_font, bg="white", fg=pink_font_color)
title_label.pack(pady=10)

# Créer un frame pour les champs de saisie
frame = tk.Frame(root, bg="#FAE8ED")
frame.pack(pady=20, padx=20)

# Créer et placer les champs de saisie
entries = []
for i, feature in enumerate(features):
    row = i // 3
    col = i % 3
    label = tk.Label(frame, text=feature, font=bold_font, fg=pink_font_color, bg="#FAE8ED")
    label.grid(row=row, column=col*2, padx=5, pady=5, sticky='e')
    entry = tk.Entry(frame, highlightbackground=pink_font_color, highlightcolor=pink_font_color, highlightthickness=1)
    entry.grid(row=row, column=col*2 + 1, padx=5, pady=5)
    entries.append(entry)

# Créer le bouton pour importer un fichier
import_button = tk.Button(frame, text="Import Values", command=load_values_from_file, font=bold_font1, fg="white", bg=pink_font_color)
import_button.grid(row=len(features)//3 + 1, column=3, pady=10, sticky='w', columnspan=2)

# Créer et placer le bouton de prédiction
predict_button = tk.Button(frame, text="Predict", command=predict_cancer, font=bold_font1, fg="white", bg=pink_font_color)
predict_button.grid(row=len(features)//3 + 1, column=4, pady=10, sticky='w', columnspan=2)



# Créer et placer le label pour afficher le résultat de la prédiction
result_label = tk.Label(root, text="", font=bold_font, bg="white", fg=pink_font_color)
result_label.pack(pady=10)



# Exécuter l'application
root.mainloop()
